In [1]:
from selenium import webdriver
from selenium.webdriver.common.keys import Keys
from webdriver_manager.chrome import ChromeDriverManager
from selenium.webdriver.chrome.service import Service
from bs4 import BeautifulSoup
from selenium.webdriver.support.ui import WebDriverWait
from selenium.webdriver.support import expected_conditions as EC
from selenium.webdriver.common.by import By
import time 
import pandas as pd
import re
import numpy as np

In [2]:
df = pd.read_csv('dataframe.csv',delimiter=';')
df2 = pd.concat([df['Title'],df['Artist']],axis=1)
df2 = df2.drop_duplicates(ignore_index=True)
# df2.to_csv("a_supp.csv")
# pd.set_option('display.max_rows', None)
# pd.set_option('display.max_columns', None)
# pd.set_option('display.width', 1000)
# pd.set_option('display.colheader_justify', 'center')
# pd.set_option('display.precision', 2)
# display(df2)

In [3]:
song_artists = list(zip(df2['Artist'],df2["Title"]))
len(song_artists)

557

In [4]:
def modif_artiste(a_remplace, remplacant):
    for i in range(len(song_artists)):
        if a_remplace in song_artists[i][0] and 'Lil Nas X' not in song_artists[i][0]:
            artist_list = list(song_artists[i])
            artist_list[0] = artist_list[0].replace(a_remplace, remplacant)
            song_artists[i] = tuple(artist_list)
            print(song_artists[i])
        

In [5]:
# modif_artiste("vs","")
# modif_artiste("Coolio Featuring L.V.","Coolio")
# modif_artiste("With"," ")
# modif_artiste("Quevedo & Mike Towers", "Quevedo")
# modif_artiste("X","&")

In [6]:
def format_artiste(char):
        for i in range(len(song_artists)):
            if char in song_artists[i][0] and 'Lil Nas X' not in song_artists[i][0]:
                artist_list = list(song_artists[i])
                artist_list[0] = artist_list[0][:artist_list[0].index(char)]
                song_artists[i] = tuple(artist_list)
                print(song_artists[i])


In [7]:
format_artiste(" vs. ")
format_artiste(" With ")
format_artiste(" X ")
format_artiste(" x ")
format_artiste(" & ")
format_artiste(" Featuring ")
format_artiste(" And ")
format_artiste(" E ")

('Lana Del Rey', 'Summertime Sadness')
('Kane Brown', 'Thank God')
('Perry Como And The Fontane Sisters', "It's Beginning To Look Like Christmas")
('John & Yoko/The Plastic Ono Band', 'Happy Xmas (War Is Over)')
('Eartha Kitt', 'Santa Baby')
('Perry Como', "(There's No Place Like) Home For The Holidays (1959)")
('RM', 'Wild Flower')
('Frank Sinatra', 'Let It Snow, Let It Snow, Let It Snow')
('RM', 'Still Life')
('Imagine Dragons', 'Enemy')
('Becky G', 'MAMIII')
('Carin Leon', 'Que Vuelvas')
('Fuerza Regida', 'Bebe Dame')
('Junior H', 'Fin de Semana')
('Fuerza Regida', 'Billete Grande')
('TOMORROW', 'Sugar Rush Ride')
('Peso Pluma', 'PRC')
('Fuerza Regida', 'Ch y La Pizza')
('Karol G', 'Gatubela')
('Coldplay', 'My Universe')
('Pablo Pesadilla', 'Baby Otaku')
('Nicki Minaj', 'Tukoh Taka (Music From The FIFA World Cup Qatar 2022)')
('Karol G', 'X Si Volvemos')
('David Guetta', "I'm Good (Blue)")
('Bad Bunny', 'Me Porto Bonito')
('Bizarrap', 'Bzrp Music Sessions, Vol. 52')
('Rema', 'Calm D

In [8]:
song_artists.pop(380)
song_artists.pop(474)
song_artists.pop(519)

df2 = df2.drop(380)
df2 = df2.drop(519)
df2 = df2.drop(474)

In [10]:
options = webdriver.ChromeOptions()
options.add_argument('--headless')
options.add_argument('--disable-gpu')
driver_genius = webdriver.Chrome(ChromeDriverManager().install(), options=options)
# driver_genius = webdriver.Chrome(ChromeDriverManager().install())
    # ouvrir le site et accepter les cookies

driver_genius.get("https://genius.com/")
# driver_genius.implicitly_wait(10)
driver_genius.find_element(By.ID,"onetrust-accept-btn-handler").click()
# time.sleep(3)
 
song,artist,date,producteurs_text,ecrivains_text,publisher_text,distributor_text,Tags = [],[],[],[],[],[],[],[]


for i in range(len(song_artists)):

    driver_genius.get("https://genius.com/")

    # activer la recherche
    element = driver_genius.find_element(By.XPATH,"/html/body/div[1]/div/div[1]/div[1]/form/input")
    element.send_keys(song_artists[i][1]+' '+song_artists[i][0], Keys.ENTER)

    # driver_genius.implicitly_wait(10)
    song.append(song_artists[i][1])
    artist.append(song_artists[i][0])
    # driver_genius.implicitly_wait(10)
    # clicker sur la chanson
    try :
        
        # driver_genius.implicitly_wait(10)
        driver_genius.find_element(By.XPATH,"/html/body/routable-page/ng-outlet/search-results-page/div/div[2]/div[1]/div[1]/search-result-section/div/div[2]/search-result-items/div/search-result-item/div/mini-song-card/a/div[2]").click()
        # driver_genius.implicitly_wait(10)
    
    except:

        # driver_genius.implicitly_wait(10)
        driver_genius.find_element(By.XPATH,"/html/body/routable-page/ng-outlet/search-results-page/div/div[2]/div[1]/div[2]/search-result-section/div/div[2]/search-result-items/div[1]/search-result-item/div/mini-song-card/a").click()
        # driver_genius.implicitly_wait(10)

    # driver_genius.implicitly_wait(10)

    #scroll to credits

    target_credits = driver_genius.find_element(By.CSS_SELECTOR,"div.SongInfo__Title-nekw6x-1")
    driver_genius.execute_script("arguments[0].scrollIntoView();", target_credits)

    # driver_genius.implicitly_wait(10)


    # Si le bouton expand est présent

    try: 
        #click on expand if this button exist

        all_elements_after_target_credit = target_credits.find_elements(By.XPATH, ".//following::*")
        # driver_genius.implicitly_wait(10)
        # trouve le 1er element dont le tag_name est "button"

        for element in all_elements_after_target_credit:
            if element.tag_name == "button" and "Expand" in element.text:
                element.click()
                break

        # driver_genius.implicitly_wait(10)
    
    except:

        # Le bouton Expand n'a pas été trouvé sur la page
        print("Le bouton n'existe pas sur la page")

    ############################################## DATE ####################################################   
    # try:

    #     # Localiser la balise "div" contenant la date en utilisant son contenu texte
    #     date_element = driver_genius.find_element(By.XPATH,"//div[contains(text(),'Release Date')]")
    #     driver_genius.implicitly_wait(10)
    #     # Extraire la date à partir du texte en utilisant des expressions régulières
    #     if date_element:
    #         date_text = date_element.find_element(By.XPATH,'./following-sibling::div').text
    #         date_regex = r"\w+\s+\d{1,2},\s+\d{4}"
    #         time.sleep(2)
    #         match = re.search(date_regex, date_text)
    #         if match:

    #             date.append(match.group())
        
    #     driver_genius.implicitly_wait(10)

    # except:

    #     date.append("None")

    # driver_genius.implicitly_wait(10)

    #Recherche des producteurs,ecrivains,publisher,distributor et tags

    ################################################## PRODUCTEURS #############################################
    try:
        producteurs_element = driver_genius.find_element(By.XPATH,"//div[contains(text(),'Produced By')]")
        producteurs_text.append(producteurs_element.find_element(By.XPATH,'./following-sibling::div').text)
    except:
        producteurs_text.append("None")
    ############################################ ECRIVAIN ######################################################
    # driver_genius.implicitly_wait(10)
    try:
        ecrivains_element = driver_genius.find_element(By.XPATH,"//div[contains(text(),'Written By')]")
        ecrivains_text.append(ecrivains_element.find_element(By.XPATH,'./following-sibling::div').text)
    except:  
        ecrivains_text.append("None")
    ####################################### PUBLISHER ##########################################################
    # driver_genius.implicitly_wait(10)
    try:
        publisher_element = driver_genius.find_element(By.XPATH,"//div[contains(text(),'Publisher')]")
        publisher_text.append(publisher_element.find_element(By.XPATH,'./following-sibling::div').text)
    except:
        publisher_text.append("None")
    ####################################### DISTIBUTEURS #######################################################
    # driver_genius.implicitly_wait(10)
    try:
        distributor_element = driver_genius.find_element(By.XPATH,"//div[contains(text(),'Distributor')]")
        distributor_text.append(distributor_element.find_element(By.XPATH,'./following-sibling::div').text)
    except:
        distributor_text.append("None")

    ############################################ TAGS ##########################################################
    # driver_genius.implicitly_wait(10)

    try:
        tag_pos = driver_genius.find_element(By.CLASS_NAME,"SongTags__Title-xixwg3-0")
        driver_genius.execute_script("arguments[0].scrollIntoView();", tag_pos)
        name = driver_genius.find_elements(By.CLASS_NAME, "SongTags__Container-xixwg3-1")

        # driver_genius.implicitly_wait(10)

        tag=[]

        for elt in name:

            tag.append(elt.text)

        tags = ''.join(set(tag))  
        
        Tags.append(tags.replace('\n',', '))
        # driver_genius.implicitly_wait(10)

    except:

        Tags.append("None")
        # driver_genius.implicitly_wait(10)
    # on revient à l'accueil
    
        driver_genius.get("https://genius.com/")
    # verif = (date[i],song[i])
    #print(date[i],song[i])
    print(i,song_artists[i][1]+' '+song_artists[i][0])
    # driver_genius.implicitly_wait(10)
    
# print(date, producteurs_text,ecrivains_text,publisher_text,distributor_text,Tags)
# df_genius = pd.DataFrame( {'Realise Date': date, 'Producers': producteurs_text, 'Writes' : ecrivains_text ,'Publisher':publisher_text,'Distributors':distributor_text, 'Tags':distributor_text})

C:\Users\camil\AppData\Local\Temp\ipykernel_13296\2683340550.py:4: DeprecationWarning: executable_path has been deprecated, please pass in a Service object
  driver_genius = webdriver.Chrome(ChromeDriverManager().install(), options=options)


0 Shut Down BLACKPINK
1 Pink Venom BLACKPINK
2 I'm Good (Blue) David Guetta
3 As It Was Harry Styles
4 Me Porto Bonito Bad Bunny
5 I Ain't Worried OneRepublic
6 Bzrp Music Sessions, Vol. 52 Bizarrap
7 Titi Me Pregunto Bad Bunny
8 Under The Influence Chris Brown
9 Super Freaky Girl Nicki Minaj
10 La Bachata Manuel Turizo
11 Despecha Rosalia
12 Calm Down Rema
13 Bad Habit Steve Lacy
14 Efecto Bad Bunny
15 Typa Girl BLACKPINK
16 Heat Waves Glass Animals
17 Snap Rosa Linn
18 Another Love Tom Odell
19 Ojitos Lindos Bad Bunny
20 Running Up That Hill (A Deal With God) Kate Bush
21 Moscow Mule Bad Bunny
22 Save Your Tears The Weeknd
23 Neverita Bad Bunny
24 Glimpse Of Us Joji
25 Sunroof Nicky Youre
26 Hard To Love BLACKPINK
27 Cold Heart (PNAU Remix) Elton John
28 I Like You (A Happier Song) Post Malone
29 Hold Me Closer Elton John
30 Provenza Karol G


In [17]:
print(len(date))
print(len(producteurs_text))
print(len(ecrivains_text))
print(len(publisher_text))
print(len(distributor_text))
print(len(Tags))

0
0
0
0
0
554


In [18]:
print(date)

[]


In [19]:
for i in range(len(date)):

    if date[i]=="Nan":

        print(i)

In [20]:
print(len(song))

554


In [149]:
list_tile = list(df2['Title'])
list_artist = list(df2['Artist'])
df_genius_557 = pd.DataFrame( { 'Title':list_tile, 'Artist':list_artist,'Genre':Tags})
df_genius_557
df_genius_557.to_csv('data_sans_doublons.csv',sep=';', index=False)

In [234]:
list_genre = ['Genre']
for value in df.iterrows():
    artist1 = value[1][1]
    title1 = value[1][0]
    # print(value[0])
    for value2 in df_genius_557.iterrows():
        artist2 = value2[1][1]
        title2 = value2[1][0]
        genre = value2[1][2]
        match_found = False
        if artist1 == artist2 and title1==title2:
            list_genre.append(genre)
            match_found = True
            break
    if not match_found:
        list_genre.append("Null")
        

In [241]:
df_finale = pd.concat([df, pd.DataFrame(list_genre,columns=['Genre'])],axis=1)
df_finale
df_finale.to_csv('dataframe_finale.csv',sep=';', index=False)